# Métodos iterativos

Nos encontramos ante la problemática de querer obtener una respuesta (aunque sea aproximada) de un sistema de ecuaciones lineales. Los métodos tradicionales nos ofrecen respuestas precisas, pero a un costo computacional bastante elevado($O(n^3)$), es por esto, que aproximaciones pueden ser bastantes útiles, y para esto es que nos dispondremos a realizar iteraciones de punto fijo, de la siguiente forma:

$$
    x_0 = dato\_inicial\\
    x_{n+1} = b - Ax_{n}
$$

Utilizaremos 3 métodos:

* Jacobi
* Gauss-Seidel
* Sor($\omega$)

Estos 3 métodos utilizan una descomposición de $A$, la cual es de la siguiente forma:

$$
    A = L + D + U
$$

Se explicara con un ejemplo para que se entienda:


$$
    A = \begin{bmatrix}
            1 & 2 & 3\\
            4 & 5 & 6\\
            7 & 8 & 9\\
        \end{bmatrix}
$$

Entonces $L$, $D$ y $U$ serán respectivamente:

$$
    L = \begin{bmatrix}
            0 & 0 & 0\\
            4 & 0 & 0\\
            7 & 8 & 0\\
        \end{bmatrix}
   ; D = \begin{bmatrix}
            1 & 0 & 0\\
            0 & 5 & 0\\
            0 & 0 & 9\\
        \end{bmatrix}
   ; U = \begin{bmatrix}
            0 & 2 & 3\\
            0 & 0 & 6\\
            0 & 0 & 0\\
        \end{bmatrix}
$$

Como veremos a continuación, Jacobi, Gauss-Seidel y Sor($\omega$) se diferencian en la forma en que realizan el despeje de la función para iterar utilizando la descomposición recién mencionada.

In [19]:
import numpy as np

def getLDU(A):
    L = np.zeros(A.shape)
    D = np.zeros(A.shape)
    U = np.zeros(A.shape)
    for i in range(A.shape[0]):
        for j in range(A.shape[1]):
            if i > j:
                L[i][j] = A[i][j]
            elif i < j:
                U[i][j] = A[i][j]
            else:
                D[i][j] = A[i][j]
    return L, D, U

## JACOBI

Queremos resolver $Ax-b=0$, lo cual puede ser escrito como $F(x) = Ax - b$,
como nos interesa encontrar solución ($F(r)=0$), entonces podemos hacer el siguiente despeje:

\begin{align}
    Ax - b &= 0\\
    Ax &= b\\
    (L+D+U)x &= b\\
    (L+U)x + Dx &= b\\
    Dx &= b - (L+U)x\\
    x &= D^{-1}(b - (L+U)x)
\end{align}

Luego renombrando a x:

$$
    x_{n+1} = D^{-1}(b - (L+U)x_n)
$$

También se puede escribir como:

$$
    x_{n+1} = x_n + D^{-1}(b - Ax_n)
$$

Definiendo $r_n = b - Ax_n$ obtenemos:

$$
    x_{n+1} = x_n + D^{-1}r_n
$$

Como podemos apreciar, nos encontramos frente a una iteración de punto fijo vectorial.

In [20]:
def jacobi(A,b, n, x_0=None):
    L, D, U = getLDU(A)
    xs = np.zeros((n,b.shape[0]))
    D_i = np.linalg.inv(D)
    LU = L+U
    if x_0 is not None:
        xs[0] = x_0
    f = lambda x, b, D_i, A: x + D_i@(b-A@x)
    for i in range(1,n):
        xs[i] = f(xs[i-1], b, D_i, A)
    return xs

In [23]:
A = np.array([
    [5 , 0,-1],
    [0 , 1, 0],
    [0 ,-3, 2]
])
b = np.array([2,2,4])
x_0 = np.array([1,1,1])
n_iter = 30

sol_jacobi = jacobi(A,b, n_iter, x_0)
sol_real = np.linalg.solve(A, b)
print("Solucion real:", sol_real, "\nSolucion del metodo:", sol_jacobi[n_iter-1])


error = np.linalg.norm(sol_real-sol_jacobi, ord=np.inf)
error, sol_jacobi

Solucion real: [1.4 2.  5. ] 
Solucion del metodo: [1.4 2.  5. ]


(5.4,
 array([[1. , 1. , 1. ],
        [0.6, 2. , 3.5],
        [1.1, 2. , 5. ],
        [1.4, 2. , 5. ],
        [1.4, 2. , 5. ],
        [1.4, 2. , 5. ],
        [1.4, 2. , 5. ],
        [1.4, 2. , 5. ],
        [1.4, 2. , 5. ],
        [1.4, 2. , 5. ],
        [1.4, 2. , 5. ],
        [1.4, 2. , 5. ],
        [1.4, 2. , 5. ],
        [1.4, 2. , 5. ],
        [1.4, 2. , 5. ],
        [1.4, 2. , 5. ],
        [1.4, 2. , 5. ],
        [1.4, 2. , 5. ],
        [1.4, 2. , 5. ],
        [1.4, 2. , 5. ],
        [1.4, 2. , 5. ],
        [1.4, 2. , 5. ],
        [1.4, 2. , 5. ],
        [1.4, 2. , 5. ],
        [1.4, 2. , 5. ],
        [1.4, 2. , 5. ],
        [1.4, 2. , 5. ],
        [1.4, 2. , 5. ],
        [1.4, 2. , 5. ],
        [1.4, 2. , 5. ]]))

# Gauss-Seidel

Continuando la idea anterior, el método de Gaus-Seidel resuelve $(L+D+U)x = b$ de la siguiente manera:

\begin{align}
    (L+D+U)x &= b\\
    (L+D)x + Ux &= b\\
    (L+D)x &= b-Ux\\
    x &= (L+D)^{-1}(b-Ux)\\
    x &= (L+D)^{-1}b - (L+D)^{-1}Ux\\
    x_{n+1} &= (L+D)^{-1}b - (L+D)^{-1}Ux_n
\end{align}

Nuevamente podemos escribir esta ecuación es usando el residuo del sistema($r_n=b-Ax$).

\begin{align}
    x_{n+1} &= (L+D)^{-1}(b-Ux_n + (L+D)x_n - (L+D)x_n)\\
    x_{n+1} &= (L+D)^{-1}(b-(L+D+U)x_n + (L+D)x_n)\\
    x_{n+1} &= (L+D)^{-1}b-(L+D)^{-1}(L+D+U)x_n + (L+D)^{-1}(L+D)x_n\\
    x_{n+1} &= (L+D)^{-1}b-(L+D)^{-1}Ax_n + x_n\\
    x_{n+1} &= x_n + (L+D)^{-1}(b-Ax_n) \\
    x_{n+1} &= x_n + (L+D)^{-1}r_n\\
\end{align}

In [6]:
def gauss_seidel(A, b, n, x_0=None):
    L, D, U = getLDU(A)
    LD = L+D
    LD_i = np.linalg.inv(LD)
    xs = np.zeros((n,b.shape[0]))
    f = lambda x, LD_i, A, b: x + LD_i@(b-A@x)
    if x_0 is not None:
        xs[0] = x_0
    for i in range(1,n):
        xs[i] = f(xs[i-1], LD_i, A, b)
    return xs

In [9]:
A = np.array([
    [12, 3,-5],
    [ 1, 5, 3],
    [ 1, 28, 76]
])
b = np.array([1,28,76])
x_0 = np.array([1,0,1])
n_iter = 20

sol_gs = gauss_seidel(A, b, n_iter, x_0)
sol_real = np.linalg.solve(A, b)
print("Solucion real:", sol_real, "\nSolucion del metodo:", sol_gs[n_iter-1])

error = np.linalg.norm(sol_real-sol_gs, ord=np.inf)
error

Solucion real: [-2.307023    6.98788067 -1.54412679] 
Solucion del metodo: [-2.307023    6.98788067 -1.54412679]


12.839030453697951

# Succesive Over-Relaxation (SOR($\omega$))

Nuevamente resolvemos $(L+D+U)x = b$, pero en este caso primero definiremos un parametro de relajación $\omega \in \mathbb{R}$. Si $\omega > 1$, entonces se refiere a _sobre-relajación_.
Luego para resolver el problema base, lo multiplicaremos por $\omega$ despejaremos de la siguiente manera:

\begin{align}
    \omega(L+D+U)x &= \omega b\\
    \omega(L+D+U)x &= \omega b & /+ Dx\\
    \omega Lx + \omega Dx + \omega Ux + Dx &= \omega b +Dx\\
    \omega Lx+ Dx &= \omega b + Dx - \omega Dx - \omega Ux \\
    (\omega L + D)x &= \omega b + (1-\omega)Dx - \omega Ux \\
    (\omega L + D)x &= \omega b + [(1-\omega)D - \omega U]x \\
    x &= (\omega L + D)^{-1}(\omega b + [(1-\omega)D - \omega U]x)\\
    x_{n+1} &= (\omega L + D)^{-1}\omega b + (\omega L + D)^{-1}[(1-\omega)D - \omega U]x_n\\
\end{align}

La versión utilizando el residuo queda de la forma:

\begin{align}
    x_{n+1} &= x_n + (L + \frac{D}{\omega})^{-1} (b-Ax_n)\\
    x_{n+1} &= x_n + (L + \frac{D}{\omega})^{-1} r_n\\
\end{align}

La demostración es trivial y se deja al lector.

Es importante notar que si $\omega = 1$, entonces estamos utilizando el método de Gauss-Seidel.

In [10]:
def sor(A, b, n, w=1, x_0=None):
    L, D, U = getLDU(A)
    LD = L+(D/w)
    LD_i = np.linalg.inv(LD)
    xs = np.zeros((n,b.shape[0]))
    f = lambda x, LD_i, A, b: x + LD_i@(b-A@x)
    if x_0 is not None:
        xs[0] = x_0
    for i in range(1,n):
        xs[i] = f(xs[i-1], LD_i, A, b)
    return xs

In [11]:
A = np.array([
    [4 , -1, -6, 0],
    [-5, -4, 10, 8],
    [0 , 9 , 4 ,-2],
    [1 , 0 , -7, 5]
])
b = np.array([2, 21, -12, -6])
x_0 = np.array([0, 0, 0, 0])
n_iter = 20
omega = 0.5

sol_sor = sor(A, b, n_iter, omega, x_0)
sol_real = np.linalg.solve(A, b)
print("Solucion real:", sol_real, "\nSolucion del metodo:", sol_sor[n_iter-1])

error = np.linalg.norm(sol_real-sol_sor, ord=np.inf)
error

Solucion real: [ 3. -2.  2.  1.] 
Solucion del metodo: [ 2.99893472 -2.00002637  1.9997894   0.99982295]


8.0

Para la obtención de $\omega$, se realizan algunas iteraciones y se analiza el error, escogiendo el que entregue el menor de estos.

### Para el ejemplo anterior:

In [12]:
def get_w(n_iter, tr, tl, pr=False):
    w_s = np.linspace(tr, tl, n_iter)
    error_w = np.zeros(n_iter)
    sol_real = np.linalg.solve(A, b)
    min_error = np.inf
    for i in range(n_iter):
        sol = sor(A, b, 4, w_s[i])
        error_w[i] = np.linalg.norm(sol_real-sol[-1], ord=np.inf)
        if pr:
            print("i: %d \t w: %f \t error: %f" % (i, w_s[i], error_w[i]))
    w = w_s[list(error_w).index(np.amin(error_w))]
    return w
n_iter = 20
tr = 0.1
tl = 2
get_w(n_iter, tr, tl, True)

i: 0 	 w: 0.100000 	 error: 2.961789
i: 1 	 w: 0.200000 	 error: 2.864461
i: 2 	 w: 0.300000 	 error: 2.409478
i: 3 	 w: 0.400000 	 error: 1.598106
i: 4 	 w: 0.500000 	 error: 0.929522
i: 5 	 w: 0.600000 	 error: 2.339834
i: 6 	 w: 0.700000 	 error: 17.841226
i: 7 	 w: 0.800000 	 error: 76.351461
i: 8 	 w: 0.900000 	 error: 250.130396
i: 9 	 w: 1.000000 	 error: 670.938448
i: 10 	 w: 1.100000 	 error: 1575.023885
i: 11 	 w: 1.200000 	 error: 3356.448142
i: 12 	 w: 1.300000 	 error: 6641.910027
i: 13 	 w: 1.400000 	 error: 12390.701288
i: 14 	 w: 1.500000 	 error: 22025.643044
i: 15 	 w: 1.600000 	 error: 37601.747722
i: 16 	 w: 1.700000 	 error: 62020.326510
i: 17 	 w: 1.800000 	 error: 99297.321057
i: 18 	 w: 1.900000 	 error: 154895.783298
i: 19 	 w: 2.000000 	 error: 236133.662000


0.5

Resolveremos los sistemas de ejemplo usados para Jacobi y Gauss-Seidel utilizando Sor($\omega$):

In [13]:
A_1 = np.array([
    [4 , 1, 3],
    [1 , 5, 1],
    [2 ,-1, 8]
])
b_1 = np.array([17,14,12])
w_1 = get_w(20, 0.1, 2)
n_iter = 20
sol_1 = sor(A_1, b_1, n_iter, w_1)
sol_1_real = np.linalg.solve(A_1, b_1)
sol_1[-1], sol_1_real

(array([3.00064045, 1.99992644, 0.99969568]), array([3., 2., 1.]))

In [14]:
A_2 = np.array([
    [12, 3,-5],
    [ 1, 5, 3],
    [ 1, 28, 76]
])
b_2 = np.array([1,28,76])
w_2 = get_w(20, 0.1, 2)
n_iter = 20
sol_2 = sor(A_2, b_2, n_iter, w_2)
sol_2_real = np.linalg.solve(A_2, b_2)
sol_2[-1], sol_2_real

(array([-2.29267512,  6.97356606, -1.53626234]),
 array([-2.307023  ,  6.98788067, -1.54412679]))

In [15]:
A_3 = np.array([
    [ 0.        , -0.11916854, -0.0964137 , -0.11484483, -0.13454229], 
    [-0.11916854,  0.02923458, -0.10880878, -0.14099307, -0.11484483],
    [-0.0964137 , -0.10880878,  0.03087862, -0.10880878, -0.0964137 ],
    [-0.11484483, -0.14099307, -0.10880878,  0.02923458, -0.11916854],
    [-0.13454229, -0.11484483, -0.0964137 , -0.11916854,  0.        ]
])
b_3 = np.array([-0.4649693664307535, -0.45458063697788376, -0.37956633726431277, -0.45458063697788376, -0.4649693664307535])
w_3 = get_w(20, 0.1, 2)
n_iter = 20
sol_3 = sor(A_3, b_3, n_iter, w_3)
sol_3_real = np.linalg.solve(A_3, b_3)
sol_3[-1], sol_3_real

LinAlgError: Singular matrix

In [16]:
(2000-2005)*(2000-2010)*(2000-2015)

-750